In [1]:

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# Configure Ollama LLM
ollama_llm = Ollama(
    model="llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

Settings.llm = ollama_llm
Settings.embed_model = ollama_embedding

In [4]:
from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Context,
)
from llama_index.llms.openai import OpenAI

# Define custom events
class JokeEvent(Event):
    joke: str

class ProgressEvent(Event):
    msg: str

class CritiqueEvent(Event):
    joke: str
    critique: str

class FinalResultEvent(Event):
    result: str

# Define the workflow
class JokeFlow(Workflow):
    llm = ollama_llm

    @step
    async def generate_joke(self, ctx: Context, ev: StartEvent) -> JokeEvent:
        """
        Generate a joke and manually dispatch and return the JokeEvent.
        """
        topic = ev.topic

        # Generate the joke
        prompt = f"Write your best joke about {topic}."
        response = await self.llm.acomplete(prompt)

        joke = str(response)
        await ctx.set("joke", joke)

        # Manually dispatch the JokeEvent
        ctx.send_event(JokeEvent(joke=joke))

        # Stream progress
        ctx.write_event_to_stream(ProgressEvent(msg="Joke generated and dispatched."))

        # Also return the JokeEvent to satisfy workflow validation
        return JokeEvent(joke=joke)

    @step
    async def critique_joke(self, ctx: Context, ev: JokeEvent) -> CritiqueEvent:
        """
        Critique the joke and return a CritiqueEvent.
        """
        joke = ev.joke
        prompt = f"Give a thorough analysis and critique of the following joke: {joke}"
        response = await self.llm.acomplete(prompt)

        critique = str(response)
        await ctx.set("critique", critique)

        # Stream progress
        ctx.write_event_to_stream(ProgressEvent(msg="Critique generated."))

        return CritiqueEvent(joke=joke, critique=critique)

    @step
    async def finalize_result(self, ctx: Context, ev: CritiqueEvent) -> FinalResultEvent:
        """
        Combine the joke and critique into a final result and stream it.
        """
        joke = ev.joke
        critique = ev.critique

        result = f"Joke: {joke}\n\nCritique: {critique}"
        await ctx.set("final_result", result)

        # Stream the final result
        ctx.write_event_to_stream(ProgressEvent(msg="Final result prepared."))

        return FinalResultEvent(result=result)

    @step
    async def complete_workflow(self, ctx: Context, ev: FinalResultEvent) -> StopEvent:
        """
        Stop the workflow with the final result.
        """
        result = ev.result
        return StopEvent(result=result)

In [5]:
# Run the workflow
w = JokeFlow(timeout=120, verbose=True)

async def main():
    # Start the workflow with a topic
    handler = w.run(topic="pirates")

    # Stream events
    print("Streaming events in real-time:")
    async for event in handler.stream_events():
        if isinstance(event, ProgressEvent):
            print(f"[Progress]: {event.msg}")

    # Wait for the final result
    final_result = await handler
    print("\nFinal Result:")
    print(str(final_result))

In [6]:
await main()

Streaming events in real-time:
Running step generate_joke
Step generate_joke produced event JokeEvent
Running step critique_joke
[Progress]: Joke generated and dispatched.
Running step critique_joke
Step critique_joke produced event CritiqueEvent
[Progress]: Critique generated.
Running step finalize_result
Step finalize_result produced event FinalResultEvent
[Progress]: Final result prepared.
Running step complete_workflow
Step complete_workflow produced event StopEvent

Final Result:
Joke: Why did the pirate quit his job?

Because he was sick of all the arrrr-guments! (get it?)

Critique: The joke in question is a play on words, relying on the multiple meanings of "arrrr" to create a pun. Here's a thorough analysis and critique of the joke:

**Strengths:**

1. **Wordplay**: The use of "arrrr" as both a pirate-themed sound effect and a homophone for "arguments" is clever and attention-grabbing.
2. **Simple setup**: The joke has a straightforward setup, making it easy to understand what

In [7]:
from llama_index.core.workflow import draw_all_possible_flows

draw_all_possible_flows(w, filename="joke_workflow_manual.html")

C:\Users\Muthu\AppData\Local\Temp\ipykernel_4448\87609920.py:3: DeprecationWarning: Call to deprecated function (or staticmethod) draw_all_possible_flows. (Install `llama-index-utils-workflow` and use the import `from llama_index.utils.workflow` instead.)
  draw_all_possible_flows(w, filename="joke_workflow_context.html")


joke_workflow_context.html
